In [ ]:
import psycopg2
import os
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from dotenv import load_dotenv
load_dotenv()
# Database connection parameters
PG_HOST = os.environ.get("PGHOST")
PG_USER = os.environ.get("PGUSER")
PG_PASSWORD = os.environ.get("PGPASSWORD")
PG_DATABASE = os.environ.get("PGDATABASE")
PG_PORT = os.environ.get("PGPORT", "5432")

def create_checkpoints_table():
    """
    Create the required table for LangGraph checkpoints in PostgreSQL
    """
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            host=PG_HOST,
            user=PG_USER,
            password=PG_PASSWORD,
            database=PG_DATABASE,
            port=PG_PORT
        )
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        
        # Create the checkpoints table
        create_table_query = """
        CREATE TABLE IF NOT EXISTS checkpoints (
            id SERIAL PRIMARY KEY,
            thread_id VARCHAR(255) NOT NULL,
            checkpoint JSONB NOT NULL,
            parent_checkpoint INTEGER REFERENCES checkpoints(id),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            metadata JSONB
        );
        """
        
        # Create index for better performance
        create_index_query = """
        CREATE INDEX IF NOT EXISTS idx_checkpoints_thread_id 
        ON checkpoints (thread_id);
        """
        
        cursor.execute(create_table_query)
        cursor.execute(create_index_query)
        
        print("✅ Checkpoints table created successfully!")
        print("✅ Index on thread_id created successfully!")
        
        # Verify the table was created
        cursor.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_name = 'checkpoints'
        """)
        
        if cursor.fetchone():
            print("✅ Table verification successful!")
        else:
            print("❌ Table creation failed!")
            
    except Exception as e:
        print(f"❌ Error creating table: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [5]:
def verify_table_structure():
    """
    Verify the table structure was created correctly
    """
    try:
        conn = psycopg2.connect(
            host=PG_HOST,
            user=PG_USER,
            password=PG_PASSWORD,
            database=PG_DATABASE,
            port=PG_PORT
        )
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT column_name, data_type, is_nullable
            FROM information_schema.columns
            WHERE table_name = 'checkpoints'
            ORDER BY ordinal_position
        """)
        
        columns = cursor.fetchall()
        print("\n📊 Table Structure:")
        print("-" * 50)
        for column in columns:
            print(f"Column: {column[0]}, Type: {column[1]}, Nullable: {column[2]}")
        
    except Exception as e:
        print(f"❌ Error verifying table structure: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    # Create the table
    create_checkpoints_table()
    
    # Verify the table structure
    verify_table_structure()

✅ Checkpoints table created successfully!
✅ Index on thread_id created successfully!
✅ Table verification successful!

📊 Table Structure:
--------------------------------------------------
Column: thread_id, Type: text, Nullable: NO
Column: checkpoint_ns, Type: text, Nullable: NO
Column: checkpoint_id, Type: text, Nullable: NO
Column: parent_checkpoint_id, Type: text, Nullable: YES
Column: type, Type: text, Nullable: YES
Column: checkpoint, Type: jsonb, Nullable: NO
Column: metadata, Type: jsonb, Nullable: NO


# Multiple table creation 

In [6]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()
# Database connection parameters
PG_HOST = os.environ.get("PGHOST")
PG_USER = os.environ.get("PGUSER")
PG_PASSWORD = os.environ.get("PGPASSWORD")
PG_DATABASE = os.environ.get("PGDATABASE")
PG_PORT = os.environ.get("PGPORT", "5432")

DB_URI = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

def create_custom_tables_manually():
    """Manually create tables with custom names using SQL"""
    try:
        conn = psycopg2.connect(DB_URI)
        conn.autocommit = True
        cursor = conn.cursor()
        
        # Create custom store table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS my_app_memories (
                key TEXT PRIMARY KEY,
                namespace TEXT NOT NULL,
                value JSONB NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        
        # Create index for better performance
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_my_app_memories_namespace 
            ON my_app_memories (namespace);
        """)
        
        # Create custom checkpoints table (simplified version)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS my_app_checkpoints (
                id SERIAL PRIMARY KEY,
                thread_id VARCHAR(255) NOT NULL,
                checkpoint JSONB NOT NULL,
                parent_checkpoint INTEGER REFERENCES my_app_checkpoints(id),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                metadata JSONB
            );
        """)
        
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_my_app_checkpoints_thread_id 
            ON my_app_checkpoints (thread_id);
        """)
        
        print("✅ Custom tables created manually!")
        
    except Exception as e:
        print(f"❌ Error creating tables manually: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Try manual approach if the LangGraph methods don't support custom names
create_custom_tables_manually()

✅ Custom tables created manually!
